In [1]:
# # read unlabelled dataset
# with open("unlabelled_dataset.txt", encoding='utf-8') as f:
#     text = f.read()

# print("Content Length: ", len(text))
# print("Top 1000: ", text[:1000])

In [2]:
import pandas as pd
df = pd.read_csv("newspeak.csv")
text = "".join([v for v in df["Newspeak Translation"]])
print(text)

Unjoy is under goodthink.The partytruth was a plusungood.He is a true goodthink.They tried to hide the unjoy.I goodthinkful my family.I goodthinkful my big brother.We must fight for partytruth.I don't believe the war.I don't believe the plusungood.Partytruth is important to me.Peace is under goodthink.The party is important to me.Ownlife is important to me.Plusungood is under goodthink.He is a true the party.We cannot trust the untruth.We must fight for untruth.I goodthinkful my family.The pastrewrite was a plusungood.We cannot trust the untruth.We must fight for peace.The now was a plusungood.She feels plusgood today.The crimestop was a plusungood.The doubleplusungood was a plusungood.Rewrite is under goodthink.Pastrewrite is under goodthink.The party is under goodthink.The partytruth was a plusungood.We cannot trust the crimestop.He is a true crimestop.Now is under goodthink.I don't believe the unperson.I don't believe the plusgood.War is under goodthink.Unjoy is under goodthink.We c

In [3]:
# # collect all unique characters from text:
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# print("vocab_size: ", vocab_size)
# print("Characters: ", "".join(chars))

In [4]:
# stoi = {ch:i for i, ch in enumerate(chars)}
# itos = {i:ch for i, ch in enumerate(chars)}
# encode = lambda s: [stoi[c] for c in s]
# decode= lambda l: ''.join([itos[i] for i in l])

# print(encode("hi there"))
# print(decode(encode("hi there")))

In [5]:
"""
Contains the base Tokenizer class and a few common helper functions.
The base class also contains the (common) save/load functionality.
It would be possible to be a lot more strict about the interface and
e.g. isolating all regex/pattern parts to the RegexTokenizer, but
some concessions are made for simplicity.
"""
import unicodedata

# -----------------------------------------------------------------------------
# a few helper functions useful for both BasicTokenizer and RegexTokenizer

def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# first two helper functions...
def replace_control_characters(s: str) -> str:
    # we don't want to print control characters
    # which distort the output (e.g. \n or much worse)
    # https://stackoverflow.com/questions/4324790/removing-control-characters-from-a-string-in-python/19016117#19016117
    # http://www.unicode.org/reports/tr44/#GC_Values_Table
    chars = []
    for ch in s:
        if unicodedata.category(ch)[0] != "C":
            chars.append(ch) # this character is ok
        else:
            chars.append(f"\\u{ord(ch):04x}") # escape
    return "".join(chars)

def render_token(t: bytes) -> str:
    # pretty print a token, escaping control characters
    s = t.decode('utf-8', errors='replace')
    s = replace_control_characters(s)
    return s

# -----------------------------------------------------------------------------
# the base Tokenizer class

class Tokenizer:
    """Base class for Tokenizers"""

    def __init__(self):
        # default: vocab size of 256 (all bytes), no merges, no patterns
        self.merges = {} # (int, int) -> int
        self.pattern = "" # str
        self.special_tokens = {} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = self._build_vocab() # int -> bytes

    def train(self, text, vocab_size, verbose=False):
        # Tokenizer can train a vocabulary of size vocab_size from text
        raise NotImplementedError

    def encode(self, text):
        # Tokenizer can encode a string into a list of integers
        raise NotImplementedError

    def decode(self, ids):
        # Tokenizer can decode a list of integers into a string
        raise NotImplementedError

    def _build_vocab(self):
        # vocab is simply and deterministically derived from merges
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def save(self, file_prefix):
        """
        Saves two files: file_prefix.vocab and file_prefix.model
        This is inspired (but not equivalent to!) sentencepiece's model saving:
        - model file is the critical one, intended for load()
        - vocab file is just a pretty printed version for human inspection only
        """
        # write the model: to be used in load() later
        model_file = file_prefix + ".model"
        with open(model_file, 'w') as f:
            # write the version, pattern and merges, that's all that's needed
            f.write("minbpe v1\n")
            f.write(f"{self.pattern}\n")
            # write the special tokens, first the number of them, then each one
            f.write(f"{len(self.special_tokens)}\n")
            for special, idx in self.special_tokens.items():
                f.write(f"{special} {idx}\n")
            # the merges dict
            for idx1, idx2 in self.merges:
                f.write(f"{idx1} {idx2}\n")
        # write the vocab: for the human to look at
        vocab_file = file_prefix + ".vocab"
        inverted_merges = {idx: pair for pair, idx in self.merges.items()}
        with open(vocab_file, "w", encoding="utf-8") as f:
            for idx, token in self.vocab.items():
                # note: many tokens may be partial utf-8 sequences
                # and cannot be decoded into valid strings. Here we're using
                # errors='replace' to replace them with the replacement char �.
                # this also means that we couldn't possibly use .vocab in load()
                # because decoding in this way is a lossy operation!
                s = render_token(token)
                # find the children of this token, if any
                if idx in inverted_merges:
                    # if this token has children, render it nicely as a merge
                    idx0, idx1 = inverted_merges[idx]
                    s0 = render_token(self.vocab[idx0])
                    s1 = render_token(self.vocab[idx1])
                    f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
                else:
                    # otherwise this is leaf token, just print it
                    # (this should just be the first 256 tokens, the bytes)
                    f.write(f"[{s}] {idx}\n")

    def load(self, model_file):
        """Inverse of save() but only for the model file"""
        assert model_file.endswith(".model")
        # read the model file
        merges = {}
        special_tokens = {}
        idx = 256
        with open(model_file, 'r', encoding="utf-8") as f:
            # read the version
            version = f.readline().strip()
            assert version == "minbpe v1"
            # read the pattern
            self.pattern = f.readline().strip()
            # read the special tokens
            num_special = int(f.readline().strip())
            for _ in range(num_special):
                special, special_idx = f.readline().strip().split()
                special_tokens[special] = int(special_idx)
            # read the merges
            for line in f:
                idx1, idx2 = map(int, line.split())
                merges[(idx1, idx2)] = idx
                idx += 1
        self.merges = merges
        self.special_tokens = special_tokens
        self.vocab = self._build_vocab()

In [6]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py

Unlike BasicTokenizer:
- RegexTokenizer handles an optional regex splitting pattern.
- RegexTokenizer handles optional special tokens.
"""

import regex as re


# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


class RegexTokenizer(Tokenizer):

    def __init__(self, pattern=None):
        """
        - pattern: optional string to override the default (GPT-4 split pattern)
        - special_tokens: str -> int dictionary of special tokens
          example: {'<|endoftext|>': 100257}
        """
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)
        self.special_tokens = {}
        self.inverse_special_tokens = {}

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # split the text up into text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
        for i in range(num_merges):
            # count the number of times every consecutive pair appears
            stats = {}
            for chunk_ids in ids:
                # passing in stats will update it in place, adding up counts
                get_stats(chunk_ids, stats)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # save class variables
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

    def register_special_tokens(self, special_tokens):
        # special_tokens is a dictionary of str -> int
        # example: {"<|endoftext|>": 100257}
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}

    def decode(self, ids):
        # given ids (list of integers), return Python string
        part_bytes = []
        for idx in ids:
            if idx in self.vocab:
                part_bytes.append(self.vocab[idx])
            elif idx in self.inverse_special_tokens:
                part_bytes.append(self.inverse_special_tokens[idx].encode("utf-8"))
            else:
                raise ValueError(f"invalid token id: {idx}")
        text_bytes = b"".join(part_bytes)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def _encode_chunk(self, text_bytes):
        # return the token ids
        # let's begin. first, convert all bytes to integers in range 0..255
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

    def encode_ordinary(self, text):
        """Encoding that ignores any special tokens."""
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8") # raw bytes
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids

    def encode(self, text, allowed_special="none_raise"):
        """
        Unlike encode_ordinary, this function handles special tokens.
        allowed_special: can be "all"|"none"|"none_raise" or a custom set of special tokens
        if none_raise, then an error is raised if any special token is encountered in text
        this is the default tiktoken behavior right now as well
        any other behavior is either annoying, or a major footgun
        """
        # decode the user desire w.r.t. handling of special tokens
        special = None
        if allowed_special == "all":
            special = self.special_tokens
        elif allowed_special == "none":
            special = {}
        elif allowed_special == "none_raise":
            special = {}
            assert all(token not in text for token in self.special_tokens)
        elif isinstance(allowed_special, set):
            special = {k: v for k, v in self.special_tokens.items() if k in allowed_special}
        else:
            raise ValueError(f"allowed_special={allowed_special} not understood")
        if not special:
            # shortcut: if no special tokens, just use the ordinary encoding
            return self.encode_ordinary(text)
        # otherwise, we have to be careful with potential special tokens in text
        # we handle special tokens by splitting the text
        # based on the occurrence of any exact match with any of the special tokens
        # we can use re.split for this. note that surrounding the pattern with ()
        # makes it into a capturing group, so the special tokens will be included
        special_pattern = "(" + "|".join(re.escape(k) for k in special) + ")"
        special_chunks = re.split(special_pattern, text)
        # now all the special characters are separated from the rest of the text
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for part in special_chunks:
            if part in special:
                # this is a special token, encode it separately as a special case
                ids.append(special[part])
            else:
                # this is an ordinary sequence, encode it normally
                ids.extend(self.encode_ordinary(part))
        return ids

In [7]:
vocab_size = 488
tokenizer = RegexTokenizer()
tokenizer.train(text, vocab_size=vocab_size, verbose=True) # 9746 is the size of the vocabulary
tokenizer.encode("hello world") # string -> tokens
tokenizer.decode([187, 287, 487]) # tokens -> string
tokenizer.save("tok400") # writes tok32k.model and tok32k.vocab
tokenizer.load("tok400.model") # loads the model back from disk

merge 1/232: (32, 116) -> 256 (b' t') had 1419 occurrences
merge 2/232: (104, 101) -> 257 (b'he') had 1049 occurrences
merge 3/232: (111, 100) -> 258 (b'od') had 923 occurrences
merge 4/232: (111, 258) -> 259 (b'ood') had 766 occurrences
merge 5/232: (103, 259) -> 260 (b'good') had 746 occurrences
merge 6/232: (117, 110) -> 261 (b'un') had 655 occurrences
merge 7/232: (117, 115) -> 262 (b'us') had 654 occurrences
merge 8/232: (32, 105) -> 263 (b' i') had 615 occurrences
merge 9/232: (116, 104) -> 264 (b'th') had 563 occurrences
merge 10/232: (32, 112) -> 265 (b' p') had 537 occurrences
merge 11/232: (256, 257) -> 266 (b' the') had 477 occurrences
merge 12/232: (32, 102) -> 267 (b' f') had 471 occurrences
merge 13/232: (263, 115) -> 268 (b' is') had 468 occurrences
merge 14/232: (105, 110) -> 269 (b'in') had 446 occurrences
merge 15/232: (269, 107) -> 270 (b'ink') had 446 occurrences
merge 16/232: (256, 114) -> 271 (b' tr') had 433 occurrences
merge 17/232: (32, 109) -> 272 (b' m') had 

In [8]:
import torch
data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
print(data.shape)

torch.Size([7473])


In [9]:
n = int(.9*len(data))
train_data = data[:n] 
val_data = data[n:]

In [10]:
context_length = 8
train_data[:context_length+1]

tensor([487, 268, 310, 276, 283, 383, 317, 280, 301])

In [11]:
x = train_data[:context_length]
y = train_data[1:context_length+1]

In [12]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLM(nn.Module):
    def __init__(self, vocab_size, n_heads, n_embed, context_length, dropout, n_layers, device):
        super().__init__()
        self.device = device
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(context_length, n_embed)
        self.blocks = nn.Sequential(
            *[Block(n_embed, n_heads, n_embed//n_heads, context_length, dropout) for _ in range(n_layers)]
        )
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both B, T
        token_embeddings = self.token_embedding_table(idx) # B, T, C
        position_embeddings = self.position_embedding_table(torch.arange(T, device=self.device)) # T, C
        x = token_embeddings + position_embeddings # B, T, C
        x = self.blocks(x) # B, T, C
        logits = self.lm_head(x) # B, T, vocab_size

        if targets is None:
            loss = None
        else:
            # reshape B, T, C
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_length:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

class Block(nn.Module):
    def __init__(self, n_embed, n_heads, head_size, context_length, dropout=0.0):
        super().__init__()
        self.self_attention_head = MultiHeadAttention(n_heads, head_size, n_embed, context_length, dropout)
        self.feed_forward = FeedForward(n_embed, dropout)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.self_attention_head(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x
    

class FeedForward(nn.Module):
    def __init__(self, n_embed, dropout=0.0):   
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size, n_embed, context_length, dropout=0.0):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embed, context_length, dropout) for _ in range(n_heads)])
        self.project_out = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.project_out(out))
        return out
    

class Head(nn.Module):
    def __init__(self, head_size, n_embed, context_length, dropout=0.0):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        

        w = q @ k.transpose(-2, -1) * C ** -0.5
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        w = F.softmax(w, dim=-1)
        w = self.dropout(w)

        v = self.value(x)
        out = w @ v
        return out

In [13]:
batch_size = 64
learning_rate = 3e-4
max_iters = 5000
eval_iters = 200
eval_interval = 500
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print("Using device: ", device)
model = BigramLM(vocab_size, n_heads=6, n_embed=384, context_length=256, dropout=0.2, n_layers=6, device=device).to(device)


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - context_length, (batch_size, ))
    x = torch.stack([data[i:i + context_length] for i in ix])
    y = torch.stack([data[i + 1:i + context_length + 1] for i in ix])
    return x.to(device), y.to(device)



@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Using device:  mps


/Users/cuburtbalanon/anaconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Step 0: train loss 6.5749, val loss 6.5873
Step 500: train loss 0.9606, val loss 1.3046
Step 1000: train loss 0.8515, val loss 1.4247
Step 1500: train loss 0.7629, val loss 1.5323
Step 2000: train loss 0.7169, val loss 1.6281
Step 2500: train loss 0.6891, val loss 1.7071
Step 3000: train loss 0.6794, val loss 1.7632
Step 3500: train loss 0.6706, val loss 1.7929
Step 4000: train loss 0.6601, val loss 1.7945
Step 4500: train loss 0.6602, val loss 1.9004


In [14]:
print(tokenizer.decode(model.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))

 .She feels plusgood today.The party is important to me.The plusungood was a plusungood.I don't believe the plusgood.The untruth was a plusungood.The plusungood was a plusungood.War is important to me.He is a true plusungood.We must fight for peace.He is a true big brother.We must fight for untruth.He is a true think.We cannot trust the think.We must fight for pastrewrite.Crimestop is under goodthink.They tried to protect the rewrite.They tried to protect the war.I don't believe the unperson.I goodthinkful my big brother.They tried to crimestop the untruth.I don't believe the unjoy.She feels unjoy today.I don't believe the big brother.I goodthinkful my big brother.He is a true goodthinkful.I don't believe the partytruth.I goodthinkful my big brother.Thoughtcrime is important to me.I don't believe the unperson.We must fight for now.The bellyfeel was a plusungood.They tried to crimestop the peace.The plusungood was a plusungood.We must fight for partytruth.The comrade was a plusungood.I 